# Monty Hall Problem Empirical Solution

## Premise of this notebook

This notebook was written as a self-assignment to train a little my skills in algorithm writing and explaining on 30/04/2023.

It aims to explain and prove empirically the solution to the Monty Hall statistical Paradox (Wikipedia page on the problem https://en.wikipedia.org/wiki/Monty_Hall_problem).

It's copyright free, you can download it from its GitHub project page at https://github.com/RAGilardi/Monty-Hall-Problem-Empirical-Solution.

This complete notebook was written in one session, so the code won't probably be completely optimized and there might be minor errors. If you have any suggestions, please write them in the issue session of the GitHub project at https://github.com/RAGilardi/Monty-Hall-Problem-Empirical-Solution/issues.

This notebook will be divided into three main sessions:
- First, I will briefly present the problem and its formal solution
- Then, I will present a commented code which will explain the logic behind my solution
- Finally, I will present an automated version of the first script which will derive the empirical demonstration.


## The problem and its explanation
Quoting Craig F. Whitaker's letter from Parade magazine (1990):
"Suppose you're on a game show, and you're given the choice of three doors:
- Behind one door is a car;
- behind the others, goats.

You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat.

He then says to you, "Do you want to pick door No. 2?" Is it to your advantage to switch your choice?"

### solution:
This problem is often referred to as a paradox as the solution is not the most intuitive:

It is statistically a safer choice to change the door after the host shows us there is a Goat behind one of the unchosen doors.

This is because with the prior knowledge we have (we know initially there were 3 possible choices and we picked a random door), we know it is still true that we had 1/3 of the chances to pick the right door and 2/3 of picking the wrong one.
Opening a wrong door doesn't change this probability and when the are presented with the opportunity to change our first pick it's just wrong to think we have a 50/50 chance of winning. We still have a 2/3 probability of having chosen the wrong door in the first place. This probability now is simply "accumulated" on the remaining door we didn't pick.

The 50/50 probability would apply just to a new player, who didn't know there were initially three closed doors. More on that in the final solution.

In my code, for the sake of simplicity, I will always let Monty Hall choose randomly which door to open for us, between the possible unpicked and not winning doors.

## Explanatory code
In this section, I will present a simple Python code with which I will explain the logic behind the algorithm.

The function is very verbose and fully commented on.

The use I made of numpy.ndarrays (3x3 matrices) is not efficient for the code but allows me to show visually the progress of the solution.

In [215]:
import numpy as np
import random as rnd
#The choice of using numpy arrays other kind of structures is completely arbitrary
#Changing it would me a slightly different choice of methods, but wouldn't affect the logic.

def testMontyHall(choice, keep):
    #the function parameters are the first door choice (from 1 to 3) and whether the player want to change it or not
    #the values for keep can be the booleans (True/False) or the string "rnd" for a random pick between the two
    
    #this section is just adding two errors whether the user choose unacceptable parameter
    validChoice = {1,2,3}
    if choice not in validChoice:
        raise ValueError("results: choice must be one of %r." % validChoice)    
    
    validKeep = {True,False}
    if keep not in validKeep:
        raise ValueError("results: keep must be one of %r." % validKeep)
                
    #changing the choice variable for readability
    choice+=-1
    
    
    ######################################## PROBLEM INITIAL SETTINGS
    
    #the code will represent the situation with a 3x3 matrix:    
    #first row show which door hides the car (1 is the car, 0 is a goat)
    #second row show the current pick (1 chosen, 0 not chosen)
    #third row show whether a door is open or not (1 open, 0 closed)
    print("Let's initialize our problem matrix:")
    doors = np.zeros(shape=(3, 3))
    print(doors)

    #randomly choose the winning door
    winner = rnd.randint(0,2)
    doors[0,winner] = 1
    print("\nAs you can see (first row of the matrix) the winning door was randomly chose to be the number",winner+1)
    print(doors)

    #pick the first choice from the function argument
    print("\nNow let's note your first pick in the second row of the matrix")
    doors[1,choice] = 1
    print(doors)

    #this array will list the other available doors which Monty Hall might open without showing you the car
    possibleSuggestions = np.array(np.arange(3))
    possibleSuggestions = possibleSuggestions[possibleSuggestions != winner]
    possibleSuggestions = possibleSuggestions[possibleSuggestions != choice]
    #print("possible suggestions: ",possibleSuggestions+1)
    
    
    ######################################## GAME

    #picking which available losing door will Monty Hall will open for you
    suggestion = rnd.choice(possibleSuggestions)
    print("\nMonty Hall will now open for you the door number: ",suggestion+1)

    print("\nAs we can see from the third row, now the door is open and we can check from first row that it hids a goat")
    doors[2,suggestion] = 1
    print(doors)

    #this integer will be used in the case of the player changing their first pick
    changeMind = np.array(np.arange(3))
    changeMind = changeMind[changeMind != suggestion]
    changeMind = changeMind[changeMind != choice]
    #print("nuova scelta:",changeMind[0]+1)

    #this section will show in function of the keep argument if the player won the car or not
    if keep:
        #we open the player first pick
        doors[2,choice] = 1
        print("\nYou chose to keep your door, so let's open it!")
        print(doors)
        if doors[0,choice] == 1:
            print("\nYou Won!")
        else:
            print("\nYou Lost!")
    else:
        doors[2,changeMind[0]] = 1
        doors[1,changeMind[0]] = 1
        doors[1,choice] = 0
        print("\nYou chose to change your door, so open the other one available!")
        print(doors)
        #a simple way to know if the player won is by counting between how many unpicked Monty Hall might have chose
        #if the player chose the right door from the beginning, Monty might have picked any of the two other doors to open
        if len(possibleSuggestions) == 2:
            print("\nYou Lost!")
        else:
            print("\nYou Won!")
            
    return 0

Now you can play a little with this function trying to see what happens changing the parameters and if you will win or not!

In [216]:
#choose between 1,2 and 3 for choice and between True, False for keep
testMontyHall(choice=1,keep=False);

Let's initialize our problem matrix:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

As you can see (first row of the matrix) the winning door was randomly chose to be the number 2
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Now let's note your first pick in the second row of the matrix
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]

Monty Hall will now open for you the door number:  3

As we can see from the third row, now the door is open and we can check from first row that it hids a goat
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]

You chose to change your door, so open the other one available!
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 1.]]

You Won!


## Empirical solution

In this final section, we will use a similar, and slightly more optimized, code iteratively, showing that with a larger number of repetitions we will have a win rate always closer:
- to 2/3 (66,7%) if we always choose to change our first pick
- to 1/3 (33,3%) if we never choose to change our first pick
- to 1/2 (50%) if we randomly choose what to do (as a player with no prior information might do)

In [217]:
import numpy as np
import random as rnd

def MontyHall(keep):
       
    #randomly chooses for you whether you keep or not the door you first picked
    if keep == "rnd":
        keep = rnd.choice([True,False])
    
    ######################################## PROBLEM INITIAL SETTINGS
        
    choice = rnd.randint(0,2) 
    
    doors = np.zeros(shape=(2, 3))

    winner = rnd.randint(0,2)
    doors[0,winner] = 1

    doors[1,choice] = 1

    possibleSuggestions = np.array(np.arange(3))
    possibleSuggestions = possibleSuggestions[possibleSuggestions != winner]
    possibleSuggestions = possibleSuggestions[possibleSuggestions != choice]
    
    result = True
    
    ######################################## GAME

    suggestion = rnd.choice(possibleSuggestions)

    changeMind = np.array(np.arange(3))
    changeMind = changeMind[changeMind != suggestion]
    changeMind = changeMind[changeMind != choice]

    if keep:
        if doors[0,choice] == 0:
            result = False
    else:
        if len(possibleSuggestions) == 2:
            result = False
            
    return result

In [218]:
def MontyHallResult(repetitions, keep):

    # ERROR CHECKS    
    if repetitions < 1:
        raise ValueError("results: the game must be run at least one time")
        
    if isinstance(repetitions, int)==False:
        raise ValueError("results: the repetition argument must be an integer")
        
    validKeep = {True,False,"rnd"}
    if keep not in validKeep:
        raise ValueError("results: keep must be one of %r." % validKeep)    
    
    # CODE
    totalWins = 0
    
    for i in range(repetitions):
        result = MontyHall(keep)
        
        if result == True:
            totalWins += 1
    
    print("Total wins: ", totalWins)
    print("Win percentage: ", totalWins/repetitions*100)

## Results
If you want to play with the final function, choose how many times you want to repeat the game and choose between True, False and "rnd"

In [222]:
#Always keep the first pick case
MontyHallResult(repetitions=10000,keep=True);

Total wins:  3389
Win percentage:  33.89


In [220]:
#Always change your pick case
MontyHallResult(repetitions=10000,keep=False);

Total wins:  6660
Win percentage:  66.60000000000001


In [221]:
#Randomly choose whether to change or not your first pick
MontyHallResult(repetitions=10000,keep="rnd");

Total wins:  4961
Win percentage:  49.61
